# Tutorial to create a PMSM Motor Environment

This example notebook aims to provide an introduction to the usage of the Gym Electric Motor toolbox.
It provides a step by step guide to the different features offered by the toolbox and its application. In this example, a guide to create a Discrete Permanent Magnet Synchronous Motor (PMSM) environment is presented.

<!-- The following code snippets are only needed if you are executing this file directly from a cloned GitHub repository where you don't have GEM installed -->

In [17]:
# from pathlib import Path
# import sys
# sys.path.append(str(Path().resolve().parent.parent))

## 1.    A Brief Introduction to GEM

The gym-electric-motor (GEM) package is a software toolbox for the simulation of different electric motors.
The toolbox is built upon the OpenAI Gym Environments for reinforcement learning. Therefore, the toolbox is specifically designed for running reinforcement learning algorithms to train agents controlling electric motors.
Beside electrical motors, also converters and load models are implemented.

The components of GEM are structured as follows:

![Motor Setup](rl_frameworks/img/SCML_Setting.svg)


### 1.1  Installation
Before you can start you need to make sure that you have gym-electric-motor installed. You can install it easily using pip:

pip install gym-electric-motor

Alternatively, you can install them and their latest developer version directly from GitHub:
https://github.com/upb-lea/gym-electric-motor


## 2.  Physical System
The system consists of a Voltage Supply, a Power Electronic Converter, an Electrical Motor and the Mechanical Load as shown in the above figure . Additionally, each SCML-System has got an ODE-Solver for the simulation.


### 2.1 Voltage Supply

The voltage supply module of the GEM toolbox provides both DC and AC voltage supplies. 
- The DC supplies privided are Ideal and Non-Ideal DC voltage sources.
- The AC supplies provided are single phase and 3-phase AC sources.
More documentation regarding the voltage supplies of GEM can be found [here.](https://upb-lea.github.io/gym-electric-motor/parts/physical_systems/voltage_supplies/voltage_supply.html)

For the PMSM environment example, let us create a non-ideal DC voltage supply:
Here, the DC-link is modeled as an RC-circuit loaded from an ideal DC voltage source. This is illustrated in the below figure. 


<img src="rl_frameworks/img/non-ideal-supply.svg" alt="non-ideal-supply" style="width: 600px;"/>

The non-ideal DC supply in GEM is named 'RCVoltageSupply' and the supply_parameter(dict) consists or Resistance R in Ohm and Capacitance C in Farad


In [18]:
import gym_electric_motor as gem

supply = 'RCVoltageSupply'
supply_parameter=dict(R=10e3, C=4e-3)  ## todo check correct values.

### 2.2 Converter and Motor
The converters are divided into two classes: The continuously controlled  and the discretely controlled converters.

In the continuous case the agent's action is controlling the converter's output voltage directly through means of duty cycling. In the discrete case the agent's action decides which converter switches are open and which are closed. Therefore, only a discrete amount of options are available. For this environment, the Discrete B6 Bridge Converter which has three switches which amounts to a total of eight possible actions is used.
<!-- ![Motor Setup](rl_frameworks/img/B6.svg) -->
<img src="rl_frameworks/img/B6.svg" alt="non-ideal-supply" style="width: 400px;"/>

The Electric motor is the Permanent Magnet Synchronous Motor.
The motor schematic is the following:


<!-- ![Motor Setup](rl_frameworks/img/ESBdq1.svg) -->
<img src="rl_frameworks/img/ESBdq1.svg" alt="non-ideal-supply" style="width: 400px;"/>

And the electrical ODEs for that motor are:

<h3 align="center">

$\frac{\mathrm{d}i_{sq}}{\mathrm{d}t} = \frac{u_{sq}-pL_d\omega_{me}i_{sd}-R_si_{sq}}{L_q}$

$\frac{\mathrm{d}i_{sd}}{\mathrm{d}t} = \frac{u_{sd}-pL_q\omega_{me}i_{sq}-R_si_{sd}}{L_d}$

$\frac{\mathrm{d}\epsilon_{el}}{\mathrm{d}t} = p\omega_{me}$

</h3>


The motor environment ID for the discrete PMSM motor is "PMSMDisc-v1". This is later passed to the 'make' function to create the motor environment. The environment IDs for other available motor environments can be found [here.](https://upb-lea.github.io/gym-electric-motor/parts/environments/environment.html)
The parameters of the specific motor is to be passed by the user as a motor parameter dictionary. Default parameters will be considered in case the motor parameters are not provided by the user. The default converter for the environment ID used here is the Discrete B6 Bridge Converter. More details can be found in the [documentation.](https://upb-lea.github.io/gym-electric-motor/parts/physical_systems/converters/B6C.html) <br>
The nominal and limit values which define the operating point of the motor is also passed as a dictionary.

In [19]:
motor_env_id = "PMSMDisc-v1"
motor_parameter = dict(p=3,  # [p] = 1, nb of pole pairs
                       r_s=17.932e-3,  # [r_s] = Ohm, stator resistance
                       l_d=0.37e-3,  # [l_d] = H, d-axis inductance
                       l_q=1.2e-3,  # [l_q] = H, q-axis inductance
                       psi_p=65.65e-3,  # [psi_p] = Vs, magnetic flux of the permanent magnet
                       )  # BRUSA

nominal_values=dict(omega=4000*2*np.pi/60,
                    i=230,
                    u=350
                    )
limit_values = {key: 1.3 * nomin for key, nomin in nominal_values.items()}


### 2.3 Motor state initializer
By Default, the motor states (e.g. motor currents, rotational speed) are always set to zero whenever the motor environment is reset. In order to make the simulation more realistic and hence, generate flexible expisodes, the motor state initializer can be used to draw the initial state values from a given probability distribution within the nominal operating range of the given motor. 

The motor_initializer is a dictionary that consists of the type of distribution, for example, uniform or gaussian distribution and the interval of values within the nominal operating region. 
Here, the motor states are initialized with values sampled from a uniform distribution from the provided intervals.



In [20]:
motor_initializer={'random_init': 'uniform', 'interval': [[-230, 230], [-230, 230], [-np.pi, np.pi]]}  
#todo check interval

### 2.4 Mechanical Load
The attached mechanical load in the toolbox is represented by the function: <br>
$ T_L(\omega) = sign(\omega_me)(c\omega_{me}^2 + sign(\omega_{me}
)b\omega_{me} +a) $  <br>
with a constant load torque a, viscous friction coefficient b and aerodynamic load torque coefficient c. These parameters as well as a moment of inertia of the load J load can be freely
defined by the user to simulate different loads.

In this example, we use the load type: "ConstSpeedLoad" which initializes the load with a constant speed at the start of each episode. The initialization value is sampled from a uniform distribution from the given interval.


In [21]:
from gym_electric_motor.physical_systems import ConstantSpeedLoad

load = 'ConstSpeedLoad'
load_initializer={'random_init': 'uniform', 'interval':[100,200]}  # todo check interval 


## 3. Reward Function
The Reward calculation is based on the current state and reference of the motor environment. It is calculated as a weighted sum of errors with a certain power follows:

<!-- <h3 align="center"> -->
$ reward = - reward\_weights * (abs(state - reference)/ state\_length)^{reward\_power}$

If states are observed for a constraint violation, an additional terminal reward is added. This value depends on the discount factor gamma as follows. 
<!-- <h3 align="center"> -->
$limit\_violation\_reward = -1 / (1 - \gamma).$


### 3.1. Constraint Monitor
The ConstraintMonitor monitors the system-states and assesses whether
    they comply the given limits or violate them.
    It returns the necessary information for the RewardFunction, to calculate
    the corresponding reward-value.
    The constraints of the system-states can be generally described by the user
    or are restricted by the physical enviroment-limits as a default constraint.
    
    
The user defined constraint: "SqdCurrentMonitor" presented below, observes the currents and raises a flag indicating a limit violation if :

$ i_{sd}^2 + i_{sq}^2 > 1 $  
Here $i_{sd}$ and $i_{sq}$ are the motor currents in the d-q coordinate system that can be accessed from the motor state.




In [22]:

gamma = 0.99  #Discount factor for the reward punishment. Should equal agents' discount factor gamma.

class SqdCurrentMonitor:
    """
    monitor for squared currents:

    i_sd**2 + i_sq**2 < 1.5 * nominal_limit
    """

    def __call__(self, state, observed_states, k, physical_system):
        self.I_SD_IDX = physical_system.state_names.index('i_sd') # access motor state i_sd
        self.I_SQ_IDX = physical_system.state_names.index('i_sq') # access motor state i_sq
        sqd_currents = state[self.I_SD_IDX] ** 2 + state[self.I_SQ_IDX] ** 2
        return sqd_currents > 1

    
reward_function=gem.reward_functions.WeightedSumOfErrors(  # The function that computes the reward
            observed_states=['i_sq', 'i_sd'],   # Names of the observed states
            reward_weights={'i_sq': 10, 'i_sd': 10}, #Reward power for each of the systems states.
            constraint_monitor=SqdCurrentMonitor(), # ConstraintMonitor for monitoring
                                                    # states regarding defined constraints
            gamma=gamma,    # Discount factor for the reward punishment. Should equal agent's 
                            # discount factor gamma.
            reward_power=1) # Reward power for each of the systems states

## 4. Reference Generator
The reference generator generates references to the observed stated, that the physical system is expected to follow. GEM toolbox provides various reference generators which can be found in the [documentation.](https://upb-lea.github.io/gym-electric-motor/parts/reference_generators/reference_generator.html)

In this example, we generate references to the motor currents $i_{sq}$ and $i_{sd}$.  <br>
The "WienerProcessReferenceGenerator" is used to generate random references for both $i_{sq}$ and $i_{sd}$. The individual sub-reference generators are then combined using the "MultipleReferenceGenerator"

In [23]:
from gym_electric_motor.reference_generators import \
    MultipleReferenceGenerator,\
    WienerProcessReferenceGenerator

q_generator = WienerProcessReferenceGenerator(reference_state='i_sq') # sub-reference generator for i_sq
d_generator = WienerProcessReferenceGenerator(reference_state='i_sd') # sub-reference generator for i_sd
rg = MultipleReferenceGenerator([q_generator, d_generator]) # combine the sub-reference generators

## 5. Visualization
The visualization module provides an interface to observe and inspect the physical system's state, references, rewards, etc. 
GEM offers two forms of visualization:
- Motor Dashboard: A graphical interface which provides visualization in the form of plots.
- Console Printer: A simpler interface in the form of prints on the console.  <br>
This example demonstrates the usage of the motor dashboard for visualization.<br>
A list of variables to be plotted is passed to the MotorDashboard during initialization. The variables that can be plotted for a given motor environment can be found [here.](https://upb-lea.github.io/gym-electric-motor/parts/visualizations/motor_dashboard.html)

In [25]:
from gym_electric_motor.visualization import MotorDashboard

visualization = MotorDashboard(plots=['i_sq', 'i_sd', 'reward']) # plots the states i_sd and i_sq and reward.

## 6. Callbacks

